In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax

In [3]:
PATH = "gdrive/My Drive/Eye_diseases"
data_dir_list = os.listdir(PATH)
data_dir_list

['cataract', 'Bulging_eyes', 'Crossed_eyes', 'Glaucoma', 'Uveitis']

In [4]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 50
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(PATH +"/"+ dataset)
    for img in img_list:
        input_img = cv2.imread(PATH +"/"+dataset+"/"+img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

Getting image from cataract folder
Getting image from Bulging_eyes folder
Getting image from Crossed_eyes folder
Getting image from Glaucoma folder
Getting image from Uveitis folder


In [5]:
num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array 
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape 
print("number of samples",num_of_samples)
print("target column before encoding",target_column)

num_classes 5
Shape of image data (383, 224, 224, 3)
number of samples 383
target column before encoding ['cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'cataract', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_eyes', 'Bulging_e

In [6]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

array([0, 1, 2, 3, 4])

In [7]:
target_column

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [8]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_temp,y_train,y_temp = train_test_split(X,Y,test_size=0.3,random_state=2)
X_test,X_val,y_test,y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=2)

In [9]:
first_Mod = Sequential()
first_Mod.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
first_Mod.add(Conv2D(64,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(Conv2D(128,(3,3),activation='relu'))
first_Mod.add(MaxPool2D(pool_size=(2,2)))
first_Mod.add(Dropout(0.2))

first_Mod.add(Flatten())
first_Mod.add(Dense(128,activation='relu'))
first_Mod.add(Dropout(0.2))
first_Mod.add(Dense(num_classes,activation='softmax'))
first_Mod.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

first_Mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

In [10]:
hist = first_Mod.fit(X_train,y_train,batch_size=batch_size,epochs=num_epoch,verbose=1,validation_data=(X_test,y_test))
score = first_Mod.evaluate(X_test,y_test,batch_size=batch_size)
print('Test Loss',score[0])
print("Test Accuracy",score[1])

Epoch 1/50
9/9 [==============================] - 4s 470ms/step - loss: 11.3507 - accuracy: 0.3955 - val_loss: 1.6938 - val_accuracy: 0.4500
Epoch 2/50
9/9 [==============================] - 3s 287ms/step - loss: 1.4941 - accuracy: 0.4478 - val_loss: 1.4559 - val_accuracy: 0.4500
Epoch 3/50
9/9 [==============================] - 3s 287ms/step - loss: 1.3935 - accuracy: 0.4627 - val_loss: 1.4569 - val_accuracy: 0.4500
Epoch 4/50
9/9 [==============================] - 3s 287ms/step - loss: 1.3655 - accuracy: 0.4664 - val_loss: 1.4323 - val_accuracy: 0.4500
Epoch 5/50
9/9 [==============================] - 3s 287ms/step - loss: 1.3960 - accuracy: 0.4291 - val_loss: 1.4514 - val_accuracy: 0.4500
Epoch 6/50
9/9 [==============================] - 3s 287ms/step - loss: 1.3751 - accuracy: 0.4664 - val_loss: 1.4269 - val_accuracy: 0.4500
Epoch 7/50
9/9 [==============================] - 3s 286ms/step - loss: 1.3319 - accuracy: 0.4701 - val_loss: 1.4430 - val_accuracy: 0.4375
Epoch 8/50
9/9 [===